In [1]:
import pandas as pd

# Load the datasets with only the required columns
df1 = pd.read_csv('C:/Users/Guest01/Documents/dataset_anime/archive/final_animedataset.csv', usecols=['user_id', 'anime_id', 'my_score', 'score', 'scored_by', 'gender', 'popularity'])
df2 = pd.read_csv('C:/Users/Guest01/Documents/dataset_anime/archive/users-score-2023.csv', usecols=['user_id', 'anime_id', 'Anime Title', 'rating'])

# Merge the datasets on user_id and anime_id
final_df = pd.merge(df2, df1, on=['user_id', 'anime_id'])

# Display the first few rows to check the data
print(final_df.head())


   user_id  anime_id             Anime Title  rating  my_score gender  score  \
0        1        21               One Piece       9         9   Male   8.54   
1        1        48             .hack//Sign       7         7   Male   7.09   
2        1       320                  A Kite       5         5   Male   6.66   
3        1        49        Aa! Megami-sama!       8         8   Male   7.38   
4        1       304  Aa! Megami-sama! Movie       8         8   Male   7.63   

   scored_by  popularity  
0     423868          35  
1      61485         650  
2      18934        1946  
3      20930        1807  
4      18571        2007  


In [2]:
# Frequency encoding for categorical columns
categorical_columns = ['gender', 'Anime Title']

for col in categorical_columns:
    freq = final_df[col].value_counts()
    final_df[col] = final_df[col].map(freq)

# Handle missing values by filling with median for numerical columns
numerical_columns = ['my_score', 'score', 'scored_by', 'popularity']
for col in numerical_columns:
    final_df[col].fillna(final_df[col].median(), inplace=True)

# Display the first few rows after preprocessing
print(final_df.head())


C:\Users\Guest01\AppData\Local\Temp\ipykernel_13520\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(final_df[col].median(), inplace=True)
C:\Users\Guest01\AppData\Local\Temp\ipykernel_13520\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

   user_id  anime_id  Anime Title  rating  my_score   gender  score  \
0        1        21        10462       9         9  4175028   8.54   
1        1        48         5341       7         7  4175028   7.09   
2        1       320         1892       5         5  4175028   6.66   
3        1        49         2448       8         8  4175028   7.38   
4        1       304         2395       8         8  4175028   7.63   

   scored_by  popularity  
0     423868          35  
1      61485         650  
2      18934        1946  
3      20930        1807  
4      18571        2007  


C:\Users\Guest01\AppData\Local\Temp\ipykernel_13520\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(final_df[col].median(), inplace=True)
C:\Users\Guest01\AppData\Local\Temp\ipykernel_13520\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [3]:
# Define features (X) and target (y)
X = final_df.drop(columns=['rating'])  # All columns except 'rating' are features
y = final_df['rating']  # 'rating' is the target variable

# Display the shapes of X and y to confirm
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (6021419, 8), y shape: (6021419,)


In [4]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [5]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np
import time
import psutil

# Function to calculate and return metrics for Gradient Boosting Regressor
def calculate_metrics(X_train, X_test, y_train, y_test):
    gb = GradientBoostingRegressor(random_state=42)

    # Optimized hyperparameters for faster execution
    # param_distributions = {
    #     'n_estimators': [30, 50],  # Reduce number of boosting stages
    #     'learning_rate': [0.01, 0.1],  # Higher learning rate to reduce need for trees
    #     'max_depth': [5, 10, 20],  # Shallower trees for faster training
    #     'min_samples_split': [2, 5, 10],  # Restrict splits to speed up
    #     'min_samples_leaf': [1, 2, 4],  # Speed up with slightly higher min samples
    #     'subsample': [0.9,1.0],  # No subsampling to avoid extra randomness
    #     'max_features': ['sqrt', 'log2']  # Use square root of features to reduce splits
    # }

    param_distributions = {
    'n_estimators': [30, 50],  # Further reduced number of trees
    'learning_rate': [0.01, 0.1],  # Balance learning rate for fewer trees
    'max_depth': [3, 5, 10, 15],  # Shallower trees for faster training
    'min_samples_split': [2, 5, 10],  # More restrictive splits
    'min_samples_leaf': [1, 2, 4],  # Fewer samples per leaf
    'subsample': [0.9, 1.0],  # No subsampling for faster processing
    'max_features': ['sqrt', 'log2']  # Use sqrt for faster splits
}

    random_search = RandomizedSearchCV(gb, param_distributions, n_iter=10, cv=3, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [6]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total       
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


Metrics for sample size 1.0:
RMSE: 0.25004607552101127
MAPE: 0.010716919305243159
R2: 0.9775029182006593
nRMSE: 0.027782897280112365
Execution Time (Raw): 2973.49453496933
Normalized Time (s/MB): 7.198968389645216
Average CPU Usage: 39.400000000000006
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 0.26847670117583367
MAPE: 0.012156498967080164
R2: 0.9740468647653377
nRMSE: 0.02983074457509263
Execution Time (Raw): 1198.8903617858887
Normalized Time (s/MB): 5.799333774499258
Average CPU Usage: 36.45
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 0.2700616085650339
MAPE: 0.012340093501108259
R2: 0.9737395403125458
nRMSE: 0.03000684539611488
Execution Time (Raw): 580.0300385951996
Normalized Time (s/MB): 5.611503799752232
Average CPU Usage: 35.75
Sample Size: 0.25
--------------------------------------------------
Metrics for sample size 0.125:
RMSE: 0.2742063277415717
MAPE: 0.

c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Metrics for sample size 100000:
RMSE: 0.2767805196947495
MAPE: 0.014974639376220933
R2: 0.9725334277661827
nRMSE: 0.030753391077194388
Execution Time (Raw): 19.7616024017334
Normalized Time (s/MB): 3.5974899305555557
Average CPU Usage: 38.2
Sample Size: 100000
--------------------------------------------------


In [7]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 84 minutes and 1.29 seconds
Total Normalized Execution Time for Entire Process: 6.45555805 seconds per MB
Total Average CPU Usage for Entire Process: 38.91%


,RMSE,MAPE,R2,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.250046,0.010717,0.977503,0.027783,2973.494535,7.198968,39.40,1.000
1,0.268477,0.012156,0.974047,0.029831,1198.890362,5.799334,36.45,0.500
2,0.270062,0.012340,0.973740,0.030007,580.030039,5.611504,35.75,0.250
3,0.274206,0.013500,0.972927,0.030467,266.219769,5.151089,40.90,0.125
4,0.836058,0.118394,0.839680,0.104507,0.164383,29.924957,41.85,100.000
5,0.344275,0.031433,0.956495,0.038253,0.370026,6.736115,44.80,1000.000
6,0.345578,0.022304,0.959725,0.038398,2.363715,4.303012,33.90,10000.000
7,0.276781,0.014975,0.972533,0.030753,19.761602,3.597490,38.20,100000.000
